# Architecture

<img src="./image.png">

# Setup

In [2]:
import sys
!{sys.executable} -m pip install sagemaker==2.42.0 -U
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip install xgboost==1.3.3

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrad

In [3]:
import boto3
import sagemaker

sess = boto3.Session()
#<!-- sm = sess.client('sagemaker') -->

# Execution role
role = sagemaker.get_execution_role()

# Session variables
region = sess.region_name
account_id = sess.client('sts', region_name=region).get_caller_identity()["Account"]

In [6]:
import sklearn 
sklearn.__version__

'0.22.1'

In [15]:
# Upload the data to S3 
import tarfile
import os.path

source_dir = 'opt/ml/model.pkl'

with tarfile.open('model.tar.gz', "w:gz") as tar:
    tar.add(source_dir, arcname=os.path.basename(source_dir))
    
bucket = 'black-belt-ml-us-east-2-257581564575'
local_model_path = "model.tar.gz"
s3uri = sagemaker.s3.S3Uploader.upload(local_model_path, f's3://{bucket}/model')
s3uri

's3://black-belt-ml-us-east-2-257581564575/model/model.tar.gz'

In [163]:
import io
import os
import json
import re
import time
from time import strftime, gmtime
from IPython import display 
from IPython.display import display as dis

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

import pprint
from time import strftime, gmtime

from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from sagemaker.inputs import TrainingInput

sm_sess = sagemaker.session.Session()
sm = boto3.client('sagemaker')

In [164]:
data_capture_prefix = 'xgb-baseline/datacapture'

endpoint_name = "xgboost-wine-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName = {}".format(endpoint_name))

EndpointName = xgboost-wine-2021-06-25-07-35-45


In [165]:
%%writefile inference.py

import os
import json
import joblib
import pandas as pd

# Same as in the training script
def model_fn(model_dir):
    """Load a model. For XGBoost Framework, a default function to load a model is not provided.
    Users should provide customized model_fn() in script.
    Args:
        model_dir: a directory where model is saved.
    Returns:
        A XGBoost model.
        XGBoost model format type.
    """
    model_files = (file for file in os.listdir(model_dir) if os.path.isfile(os.path.join(model_dir, file)))
    model_file = next(model_files)
    
    booster = joblib.load(os.path.join(model_dir, model_file))
    
    return booster 


def input_fn(request_body, request_content_type):
    """
    The SageMaker XGBoost model server receives the request data body and the content type,
    and invokes the `input_fn`.
    The input_fn that just validates request_content_type and prints
    """
    print(request_body)
    
    user_input = json.loads(request_body)["input"]

    model_input = pd.DataFrame([user_input], columns=['country', 'description', 'designation', 'price', 'province',
           'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
           'variety', 'winery'])
    
    return model_input
  

def predict_fn(input_object, model):
    """
    SageMaker XGBoost model server invokes `predict_fn` on the return value of `input_fn`.
    """
    print('Running predictions...')
    return model.predict(input_object)[0]


def output_fn(prediction, response_content_type):
    """
    After invoking predict_fn, the model server invokes `output_fn`.
    An output_fn that just adds a column to the output and validates response_content_type
    """    
    print('prediction', prediction)
    
    output = { 
                "output": round(prediction) 
             }
    
    return  json.dumps(output)
            

Overwriting inference.py


In [166]:
framework_version = '1.2-2'
docker_image_name = sagemaker.image_uris.retrieve(framework='xgboost', region='us-east-2', version=framework_version)

# Workaround while versions are not updated in SM SDK
framework_version = '1.3-1'
docker_image_name = docker_image_name[:-5] + framework_version
docker_image_name

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


'257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.3-1'

In [167]:
from sagemaker.xgboost.model import XGBoostModel

script = 'inference.py'

xgb_inference_model = XGBoostModel(
    entry_point='inference.py',
    model_data=s3uri,
    role=role,
    image_uri=docker_image_name,
    framework_version="1.3-1",
    py_version="py3"
)


predictor = xgb_inference_model.deploy(initial_instance_count=1, 
                                        instance_type='ml.m4.xlarge',
                                        endpoint_name=endpoint_name,
                                        data_capture_config=DataCaptureConfig(
                                            enable_capture=True,
                                            sampling_percentage=100,
                                            destination_s3_uri='s3://{}/{}'.format(bucket, data_capture_prefix)
                                        )
                                       )


INFO:sagemaker:Creating model with name: sagemaker-xgboost-2021-06-25-07-35-53-205
INFO:sagemaker:Creating endpoint with name xgboost-wine-2021-06-25-07-35-45


-------------!

# Cost

Deploy:
- instância: 1 x ml.m4.xlarge
- preço unitário: 0.24 USD/hora

Test de carga:
- 1000 req/min
    
Custo total (1000 reg/sec):
- 0.24 USD/hora * 60 = 14.40 USD/hora